In [ ]:
import pandas as pd

# Load CSV file
df = pd.read_csv('/content/drive/MyDrive/MSc Dissertation/Tescobank reviews.csv')
import pandas as pd

# Annotated dataset for 3-star reviews
df_annotations = pd.read_csv('/content/drive/MyDrive/MSc Dissertation/Annotated tesco1.csv')  # should have columns like: review_text, annotated_sentiment


# Inspect the data
print(df.head())


In [ ]:
df_annotations.head()

In [ ]:
df['Review Date'] = df['Review Date'].str.replace('Date of experience: ', '').str.strip()
df['Star Rating'] = df['Star Rating'].str.extract(r'(\d+)').astype(int)

In [ ]:
df['Review Date'] = pd.to_datetime(df['Review Date'], errors='coerce')

In [ ]:
df_annotations['Date'] = pd.to_datetime(df_annotations['Date'], errors='coerce')

In [ ]:
df = df.dropna(subset=['Review Message'])

In [ ]:
df_annotations  = df_annotations.dropna(subset=['Review message'])

In [ ]:
# Remove duplicates
df = df.drop_duplicates()

In [ ]:
df_annotations  = df_annotations.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df_annotations.info()

In [ ]:
df['Author Name'] = df['Author Name'].fillna('Anonymous')

In [ ]:
c = df[df['Review Date'].dt.year.between(2020, 2025)]

In [ ]:
c = df_annotations[df_annotations['Date'].dt.year.between(2020, 2025)]

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt

df['Star Rating'].value_counts().sort_index().plot(
    kind='bar', color='skyblue', edgecolor='black'
)
plt.title('Tesco Bank - Distribution of Star Ratings')
plt.xlabel('Star Rating')
plt.ylabel('Count of Reviews')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
def rate_to_sentiment(star):
    if star <= 2:
        return 'Negative'
    elif star == 3:
        return 'Neutral'
    else:
        return 'Positive'

df['Sentiment Label'] = df['Star Rating'].apply(rate_to_sentiment)



In [ ]:
import matplotlib.pyplot as plt

# Plot sentiment label distribution
df['Sentiment Label'].value_counts().plot(
    kind='bar',
    color=['green', 'red', 'grey'],  # red = Negative, gray = Neutral, green = Positive
    edgecolor='black'
)

plt.title('Tesco Bank Sentiment Distribution from Star Ratings')
plt.xlabel('Sentiment Label')
plt.ylabel('Number of Reviews')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [ ]:
!pip install --upgrade --force-reinstall numpy
!pip install --upgrade gensim

In [ ]:
from gensim.utils import simple_preprocess

In [ ]:
# Cleaning function
def clean_text(text):
    doc = nlp(str(text).lower())  # Ensure text is string
    return [
        token.lemma_ for token in doc
        if token.is_alpha and not token.is_stop and len(token) > 2
    ]

# Apply cleaning
df['cleaned_text'] = df['Review Message'].apply(clean_text)

# Inspect cleaned data
print(df[['Review Message', 'cleaned_text']].head())


In [ ]:
!pip install gensim

In [ ]:
!pip install scipy==1.13.1


In [ ]:
pip install --force-reinstall gensim

In [ ]:
import gensim
print(gensim.__version__)

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
# Build bigram model
bigram = Phrases(df['cleaned_text'], min_count=5, threshold=10)
bigram_mod = Phraser(bigram)

# Apply bigram model
df['cleaned_text'] = df['cleaned_text'].apply(lambda text: bigram_mod[text])

In [ ]:
from gensim.corpora import Dictionary

# Create dictionary
id2word = Dictionary(df['cleaned_text'])

# Filter extremes (optional, for better topic quality)
id2word.filter_extremes(no_below=5, no_above=0.6)

# Create corpus (bag of words)
corpus = [id2word.doc2bow(text) for text in df['cleaned_text']]


In [ ]:
from gensim.models import LdaModel

# Train LDA model
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=5,         # Tune this value based on coherence score
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
)

# Print the topics
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}: {topic}\n")


In [ ]:
from gensim.models import CoherenceModel

# Calculate coherence score
coherence_model_lda = CoherenceModel(
    model=lda_model,
    texts=df['cleaned_text'],
    dictionary=id2word,
    coherence='c_v'
)

coherence_score = coherence_model_lda.get_coherence()
print(f"Coherence Score: {coherence_score}")


In [ ]:
topic_labels = {
    0: "Insurance Claim Issues",
    1: "Account & Customer Support",
    2: "Credit Card Issues",
    3: "Customer Service Quality",
    4: "Insurance Coverage & Experience"
}

In [ ]:
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx} - {topic_labels[idx]}: {topic}\n")


In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Apply sentiment analysis to each review
df["Sentiment Score"] = df["Review Message"].apply(lambda text: sia.polarity_scores(text)["compound"])

# Assign topics to each review
df["Topic"] = [lda_model.get_document_topics(corpus[i], minimum_probability=0.3) for i in range(len(df))]

# Extract dominant topic per review and map it to topic name
df["Dominant Topic"] = df["Topic"].apply(lambda topics: topic_labels[max(topics, key=lambda x: x[1])[0]] if topics else None)

# Get average sentiment scores per topic
sentiment_by_topic = df.groupby("Dominant Topic")["Sentiment Score"].mean()

print("\n✅ Average Sentiment Score per Topic:")
print(sentiment_by_topic)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Sort sentiment scores for better visualization
sentiment_by_topic = sentiment_by_topic.sort_values(ascending=True)

# Create a horizontal bar chart with all bars in green
plt.figure(figsize=(10, 6))
sns.barplot(x=sentiment_by_topic.values, y=sentiment_by_topic.index, color="green")

# Labels and title
plt.xlabel("Average Sentiment Score")
plt.ylabel("Topic")
plt.title("Sentiment Score per Topic")
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)  # Vertical line at 0

# Remove legend (not needed when all bars are green)
plt.legend([],[], frameon=False)

# Show plot
plt.show()


In [ ]:
# Count the number of reviews per topic
topic_distribution = df["Dominant Topic"].value_counts()

# Create a horizontal bar chart for topic distribution with blue bars
plt.figure(figsize=(10, 6))
sns.barplot(x=topic_distribution.values, y=topic_distribution.index, color="blue")

# Labels and title
plt.xlabel("Number of Reviews")
plt.ylabel("Topic")
plt.title("Distribution of Topics")

# Show plot
plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# Combine all reviews into a single string
all_reviews_text = " ".join(df["Review Message"].astype(str))

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color="white", colormap="viridis").generate(all_reviews_text)

# Plot the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")  # Hide axes for better visualization
plt.title("Word Cloud of All Reviews - Tesco Bank", fontsize=14)
plt.show()

In [ ]:
# Define sentiment categories
df["Sentiment Category"] = df["Sentiment Score"].apply(lambda x: "Positive" if x > 0.05 else ("Negative" if x < -0.05 else "Neutral"))

# Calculate sentiment distribution per topic
sentiment_distribution = df.groupby("Dominant Topic")["Sentiment Category"].value_counts(normalize=True).unstack()

# Create a horizontal stacked bar chart
plt.figure(figsize=(12, 6))
sentiment_distribution.plot(kind="barh", stacked=True, colormap="coolwarm", alpha=0.85, figsize=(12, 6))

# Labels and title
plt.ylabel("Topic", fontsize=12)
plt.xlabel("Proportion of Sentiment", fontsize=12)
plt.title("Sentiment Distribution by Topics", fontsize=14)
plt.xticks(rotation=0)  # Keep x-axis labels horizontal
plt.legend(title="Sentiment Category")

# Show plot
plt.show()

In [ ]:
# Define sentiment categories
df["Sentiment Category"] = df["Sentiment Score"].apply(lambda x: "Positive" if x > 0.05 else ("Negative" if x < -0.05 else "Neutral"))

# Calculate sentiment distribution per topic
sentiment_distribution = df.groupby("Dominant Topic")["Sentiment Category"].value_counts(normalize=True).unstack()

# Define custom color mapping: Negative = red, Neutral = grey, Positive = blue
custom_colors = {
    "Negative": "#d62728",  # red
    "Neutral": "#d3d3d3",   # light grey
    "Positive": "#1f77b4"   # blue
}

# Create a horizontal stacked bar chart
plt.figure(figsize=(12, 6))
sentiment_distribution.plot(
    kind="barh",
    stacked=True,
    color=[custom_colors[col] for col in sentiment_distribution.columns],  # apply custom colors
    alpha=0.85,
    figsize=(12, 6)
)

# Labels and title
plt.ylabel("Topic", fontsize=12)
plt.xlabel("Proportion of Sentiment", fontsize=12)
plt.title("Sentiment Distribution by Topics", fontsize=14)
plt.xticks(rotation=0)
plt.legend(title="Sentiment Category", loc="best")

# Show plot
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
# Ensure both dictionaries have the same order
topics = list(topic_distribution.keys())
review_counts = np.array([topic_distribution[topic] for topic in topics])
sentiment_scores = np.array([sentiment_by_topic[topic] for topic in topics])

# Create figure and axis
fig, ax1 = plt.subplots(figsize=(12, 6))

# Define bar positions (y-coordinates)
y_pos = np.arange(len(topics))

# Plot topic distribution (Blue Bars - Reviews)
ax1.barh(y_pos - 0.2, review_counts, color="lightblue", height=0.4, label="Number of Reviews")
ax1.set_xlabel("Number of Reviews", color="blue", fontsize=12)
ax1.set_ylabel("Topic", fontsize=12)
ax1.set_yticks(y_pos)
ax1.set_yticklabels(topics)
ax1.tick_params(axis="x", labelcolor="blue")

# Create second axis for sentiment scores
ax2 = ax1.twiny()

# Plot sentiment scores (Red Bars - Sentiment)
ax2.barh(y_pos + 0.2, sentiment_scores, color="lightcoral", height=0.4, label="Average Sentiment")
ax2.set_xlabel("Average Sentiment Score", color="red", fontsize=12)
ax2.tick_params(axis="x", labelcolor="red")

# Add title
plt.title("Topic Distribution and Average Sentiment", fontsize=14)

# Add legends at different positions
ax1.legend(loc="lower right", fontsize=10)
ax2.legend(loc="upper right", fontsize=10)

# Show plot
plt.show()

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Visualize the topics
vis = gensimvis.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis)

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def get_vader_sentiment(text):
    score = analyzer.polarity_scores(str(text))['compound']
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df['vader_sentiment'] = df['Review Message'].apply(get_vader_sentiment)


In [ ]:
# Merge the manual sentiment annotations into the main dataset (on review text)
# Example: Merging on 'review_text' in df_reviews and 'review' in df_annotations
df_merged = df.merge(df_annotations[['Review message', 'Annotation']],
                             left_on='Review Message', right_on='Review message', how='left')


In [ ]:
df_merged = df_merged.drop(columns=['Review message'])

In [ ]:
df_merged.head()

In [ ]:
# Assign expected sentiment based on annotation or star rating
def define_expected_sentiment(row):
    if pd.notnull(row['Annotation']):
        return row['Annotation']  # Use manual label for 3-star
    elif row['Star Rating'] in [1, 2]:
        return 'negative'
    elif row['Star Rating'] in [4, 5]:
        return 'positive'
    else:
        return 'neutral'  # fallback for unexpected cases

df_merged['expected_sentiment'] = df_merged.apply(define_expected_sentiment, axis=1)


In [ ]:
df_merged.head()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

y_true = df_merged['expected_sentiment']
y_pred = df_merged['vader_sentiment']

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Detailed report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['negative', 'neutral', 'positive']))


In [ ]:
print("y_true labels:", y_true.unique())
print("y_pred labels:", y_pred.unique())

In [ ]:
# Step 1: Define a manual correction map for typos and inconsistencies
label_corrections = {
    'positive': 'positive',
    'Positive': 'positive',
    ' negative': 'negative',
    'Negative': 'negative',
    'negative': 'negative',
    ' neutral': 'neutral',
    'Neutral': 'neutral',
    'neutral': 'neutral',
    'Neutal': 'neutral'  # typo correction
}

# Step 2: Apply correction to y_true (expected_sentiment column)
df_merged['expected_sentiment'] = df_merged['expected_sentiment'].map(label_corrections)

# Step 3: Filter to keep only valid, non-null rows
valid_sentiments = ['negative', 'neutral', 'positive']
df_clean = df_merged[
    df_merged['expected_sentiment'].isin(valid_sentiments) &
    df_merged['vader_sentiment'].isin(valid_sentiments)
]

# Redefine y_true and y_pred
y_true = df_clean['expected_sentiment']
y_pred = df_clean['vader_sentiment']


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['negative', 'neutral', 'positive'], zero_division=0))


In [ ]:
!pip install transformers torch emoji


In [ ]:
import re

# Define a lightweight cleaning function
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # Remove URLs
    text = re.sub(r'\s+', ' ', text).strip()             # Remove extra whitespace
    text = text.replace('\n', ' ').replace('\r', ' ')    # Flatten line breaks
    return text

# Apply to your dataset
df_merged['Review Message'] = df_clean['Review Message'].apply(clean_text)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Mapping output class to sentiment
id2label = {0: 'negative', 1: 'neutral', 2: 'positive'}

def batch_bert_sentiment(df_merged, batch_size=32):
    texts = list(df_merged['Review Message'].astype(str))  # Ensure strings

    # Tokenize all at once
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

    # Move inputs to GPU
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Batch input
    dataset = TensorDataset(input_ids, attention_mask)
    dataloader = DataLoader(dataset, batch_size=batch_size)

    # Predict in batches
    predictions = []
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            ids, masks = batch
            outputs = model(input_ids=ids, attention_mask=masks)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(preds)

    # Convert to sentiment labels
    return [id2label[pred] for pred in predictions]


In [ ]:
# Run the sentiment prediction — fast and efficient!
df_merged['bert_sentiment'] = batch_bert_sentiment(df_merged)

In [ ]:
print("y_true classes:", sorted(y_true.unique()))
print("y_pred classes:", sorted(y_pred.unique()))

In [ ]:
# Clean and filter invalid entries
valid_labels = ['negative', 'neutral', 'positive']

# Drop rows with 'nan' (string) or actual NaN in y_true or y_pred
df_cleaned = df_merged[
    df_merged['expected_sentiment'].astype(str).str.strip().str.lower().isin(valid_labels) &
    df_merged['bert_sentiment'].astype(str).str.strip().str.lower().isin(valid_labels)
].copy()

# Standardize the labels
df_cleaned['expected_sentiment'] = df_cleaned['expected_sentiment'].str.strip().str.lower()
df_cleaned['bert_sentiment'] = df_cleaned['bert_sentiment'].str.strip().str.lower()

# Redefine y_true and y_pred
y_true = df_cleaned['expected_sentiment']
y_pred = df_cleaned['bert_sentiment']


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:")
print(classification_report(
    y_true, y_pred,
    target_names=['negative', 'neutral', 'positive'],
    zero_division=0
))


In [ ]:
import torch.nn.functional as F

def batch_bert_sentiment_scores(df_merged, batch_size=32):
    texts = list(df_merged['Review Message'].astype(str))  # Ensure strings

    # Tokenize all at once
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

    # Move inputs to GPU
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Batch input
    dataset = TensorDataset(input_ids, attention_mask)
    dataloader = DataLoader(dataset, batch_size=batch_size)

    # Predict in batches
    predictions = []
    scores = []

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            ids, masks = batch
            outputs = model(input_ids=ids, attention_mask=masks)
            logits = outputs.logits
            probs = F.softmax(logits, dim=1).cpu().numpy()
            preds = np.argmax(probs, axis=1)
            predictions.extend(preds)

            # You can pick the confidence of the predicted label
            predicted_scores = probs[np.arange(len(probs)), preds]
            scores.extend(predicted_scores)

    # Convert to sentiment labels
    labels = [id2label[pred] for pred in predictions]

    return labels, scores


In [ ]:
# Run prediction
labels, scores = batch_bert_sentiment_scores(df_merged)

# Add to your dataframe
df_merged['bert_sentiment'] = labels
df_merged['bert_sentiment_score'] = scores


In [ ]:
df_merged.head(20)

In [ ]:
df_negative = df[df['Star Rating'].isin([1, 2, 3])]

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

# Combine all review messages into one string
text_negative = " ".join(review for review in df_negative['Review Message'].astype(str))

# Define additional stopwords if needed
custom_stopwords = set(STOPWORDS)

# Create the word cloud
wordcloud_negative = WordCloud(width=800, height=400,
                               background_color='white',
                               stopwords=custom_stopwords,
                               colormap='Reds').generate(text_negative)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_negative, interpolation='bilinear')
plt.axis("off")
plt.title("Word Cloud of Negative Reviews – Tesco Bank")
plt.tight_layout()
plt.show()


In [ ]:
df_negative.head()

In [ ]:
# Create corpus for df_negative using the existing id2word dictionary
corpus_negative = [id2word.doc2bow(text) for text in df_negative['cleaned_text']]


In [ ]:
from gensim.models import LdaModel

lda_model_negative = LdaModel(corpus=corpus_negative,
                              id2word=id2word,
                              num_topics=3,  # or however many you want
                              random_state=42,
                              passes=10,
                              per_word_topics=True)

# Display topics
for idx, topic in lda_model_negative.print_topics(-1):
    print(f"Topic {idx}: {topic}")

In [ ]:

# Compute Coherence Score
coherence_model_negative = CoherenceModel(
    model=lda_model_negative,
    texts=df_negative['cleaned_text'],  # this should be tokenized text (list of words)
    dictionary=id2word,
    coherence='c_v'  # 'c_v' is a popular choice, but others are available
)

coherence_score = coherence_model_negative.get_coherence()
print(f"\nCoherence Score: {coherence_score:.4f}")


In [ ]:
topic_labels = {
    0: "Credit card Issues",
    1: "Customer Service Complaints",
    2: "Insurance Claims & Policy Issues"
}


In [ ]:
print(df_negative.columns)


In [ ]:
topic_labels = {
    0: "Credit card Issues",
    1: "Customer Service Complaints",
    2: "Insurance Claims & Policy Issues"
}

# 2. Function to get dominant topic index
def get_dominant_topic(ldamodel, corpus):
    dominant_topics = []
    for bow in corpus:
        topic_probs = ldamodel.get_document_topics(bow)
        if topic_probs:
            dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
            dominant_topics.append(dominant_topic)
        else:
            dominant_topics.append(None)
    return dominant_topics

# 3. Apply to negative reviews
df_negative['Theme'] = get_dominant_topic(lda_model_negative, corpus_negative)

# 4. Create final DataFrame with the columns you want
df_negative_selected = df_negative[['cleaned_text', 'Sentiment Score', 'Theme']].copy()

# 5. Map the topic labels as 'Theme Label'
df_negative_selected['Theme Label'] = df_negative_selected['Theme'].map(topic_labels)

# 6. Done! Preview your data
print(df_negative_selected.head())


In [ ]:
df_negative.loc[:, 'Theme Label'] = df_negative['Theme'].map(topic_labels)

In [ ]:
# Step 3: Select the key columns
df_negative_selected = df_negative[['cleaned_text', 'Sentiment Score', 'Theme Label']].copy()

In [ ]:
df_negative_selected.head()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Join all words in cleaned_text into a single string
text = ' '.join([' '.join(words) for words in df_negative_selected['cleaned_text'] if isinstance(words, list)])

# Create and display the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud for Negative Reviews")
plt.show()


In [ ]:
# Count the number of reviews per Theme Label
theme_counts = df_negative_selected['Theme Label'].value_counts().reset_index()
theme_counts.columns = ['Theme Label', 'Count']
print(theme_counts)

# Horizontal bar chart with red bars
plt.figure(figsize=(10, 6))
plt.barh(theme_counts['Theme Label'], theme_counts['Count'], color='red')
plt.title('Number of Reviews per Theme Label')
plt.xlabel('Count')
plt.ylabel('Theme Label')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Load CSV file
df_store = pd.read_csv('/content/drive/MyDrive/MSc Dissertation/Tescostore (2).csv')

In [ ]:
df_store.head()

In [ ]:
df_store['Review Date'] = df_store['Review Date'].str.replace('Date of experience: ', '').str.strip()
df_store['Star Rating'] = df_store['Star Rating'].str.extract(r'(\d+)').astype('Int64')

In [ ]:
df_store['Review Date'] = pd.to_datetime(df_store['Review Date'], errors='coerce')


In [ ]:
df_store = df_store.dropna(subset=['Review Message'])

In [ ]:
# Remove duplicates
df_store = df_store.drop_duplicates()

In [ ]:
df_annotations['Date'] = pd.to_datetime(df_annotations['Date'], errors='coerce')

In [ ]:
df_store['Author Name'] = df_store['Author Name'].fillna('Anonymous')

In [ ]:
c = df_store[df_store['Review Date'].dt.year.between(2020, 2025)]

In [ ]:
df_store.info()

In [ ]:
# Show rows where Review Date is missing
print(df_store[df_store['Review Date'].isna()])

# Show rows where Star Rating is missing
print(df_store[df_store['Star Rating'].isna()])


In [ ]:
# Drop row with index 12237
df_store = df_store.drop(index=12237).reset_index(drop=True)


In [ ]:
import matplotlib.pyplot as plt

df_store['Star Rating'].value_counts().sort_index().plot(
    kind='bar', color='orange', edgecolor='black'
)
plt.title('Tesco Stores - Distribution of Star Ratings')
plt.xlabel('Star Rating')
plt.ylabel('Count of Reviews')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
def rate_to_sentiment(star):
    if star <= 2:
        return 'Negative'
    elif star == 3:
        return 'Neutral'
    else:
        return 'Positive'

df_store['Sentiment Label'] = df_store['Star Rating'].apply(rate_to_sentiment)



In [ ]:
import matplotlib.pyplot as plt

# Plot sentiment label distribution
df_store['Sentiment Label'].value_counts().plot(
    kind='bar',
    color=['red', 'green', 'gray'],  # red = Negative, gray = Neutral, green = Positive
    edgecolor='black'
)

plt.title('Tesco Stores - Sentiment Distribution from Star Ratings')
plt.xlabel('Sentiment Label')
plt.ylabel('Number of Reviews')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# Cleaning function
def clean_text(text):
    doc = nlp(str(text).lower())  # Ensure text is string
    return [
        token.lemma_ for token in doc
        if token.is_alpha and not token.is_stop and len(token) > 2
    ]

# Apply cleaning
df_store['cleaned_text'] = df_store['Review Message'].apply(clean_text)

# Inspect cleaned data
print(df_store[['Review Message', 'cleaned_text']].head())

In [ ]:
# Tokenize and lowercase, remove punctuation
tokenized_texts = df_store['Review Message'].dropna().apply(
    lambda x: simple_preprocess(str(x), deacc=True)
).tolist()


In [ ]:
# Build bigram model
bigram = Phrases(df_store['cleaned_text'], min_count=5, threshold=10)
bigram_mod = Phraser(bigram)

# Apply bigram model
df_store['cleaned_text'] = df_store['cleaned_text'].apply(lambda text: bigram_mod[text])

In [ ]:
import nltk

# Download essential tokenizers
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Apply stopword filtering
tokenized_texts = df_store['Review Message'].dropna().apply(
    lambda x: [word for word in simple_preprocess(str(x), deacc=True) if word not in stop_words]
).tolist()


In [ ]:

# Build the bigram and trigram models
bigram = Phrases(tokenized_texts, min_count=5, threshold=100)
trigram = Phrases(bigram[tokenized_texts], threshold=100)

# Faster transformation
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

# Apply bigrams and trigrams
texts_bigrams = [bigram_mod[doc] for doc in tokenized_texts]
texts_trigrams = [trigram_mod[bigram_mod[doc]] for doc in tokenized_texts]


In [ ]:
# Lemmatize while keeping only nouns, adjectives, verbs, adverbs
def lemmatize_texts(texts):
    lemmatized_texts = []
    for doc in texts:
        joined = ' '.join(doc)  # Convert list to string
        spacy_doc = nlp(joined)
        lemmatized_texts.append([token.lemma_ for token in spacy_doc if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']])
    return lemmatized_texts

texts_lemmatized = lemmatize_texts(texts_trigrams)


In [ ]:
from gensim.corpora import Dictionary

# Create Dictionary and Corpus for Tesco Stores
id2word_stores = Dictionary(texts_lemmatized)
corpus_stores = [id2word_stores.doc2bow(text) for text in texts_lemmatized]


In [ ]:
from gensim.models import LdaModel

lda_model_stores = LdaModel(
    corpus=corpus_stores,
    id2word=id2word_stores,
    num_topics=6,               # You can still tune this
    random_state=42,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
)


In [ ]:
topics = lda_model_stores.print_topics(num_words=10)
for idx, topic in topics:
    print(f"Theme {idx}: {topic}")


In [ ]:
from gensim.models import CoherenceModel

coherence_model_stores = CoherenceModel(
    model=lda_model_stores,
    texts=texts_lemmatized,   # your final tokenized store reviews
    dictionary=id2word,       # if you've renamed it, use id2word_stores
    coherence='c_v'
)
coherence_score_stores = coherence_model_stores.get_coherence()
print(f"\nTesco Stores Coherence Score: {coherence_score_stores:.4f}")


In [ ]:
store_themes = {
0: "Delivery Problem",
1: "Product Quality and Value",
2: "Shopping at Tesco/Billing",
3: "Customer Service Interactions",
4: "In-Store Experience",
5: "Alternative Options and Comparisons"
}

In [ ]:
for idx, topic in lda_model_stores.print_topics():
    print(f"Theme {idx} - {store_themes[idx]}: {topic}\n")


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Step 1: Apply sentiment analysis to each review
df_store["Sentiment Score"] = df_store["Review Message"].apply(
    lambda text: sia.polarity_scores(text)["compound"]
)

# Step 2: Assign topic distribution from the LDA model
df_store["Topic"] = [
    lda_model_stores.get_document_topics(corpus_stores[i], minimum_probability=0.3)
    for i in range(len(df_store))
]

# Step 3: Extract dominant topic and map to store theme label
df_store["Dominant Topic"] = df_store["Topic"].apply(
    lambda topics: store_themes[max(topics, key=lambda x: x[1])[0]] if topics else None
)

# Step 4: Calculate average sentiment per topic
sentiment_by_topic_store = df_store.groupby("Dominant Topic")["Sentiment Score"].mean()

# Display result
print("\n✅ Average Sentiment Score per Tesco Store Topic:")
print(sentiment_by_topic_store)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Sort the sentiment scores
sentiment_by_topic_store = sentiment_by_topic_store.sort_values(ascending=True)

# Generate color list based on sentiment polarity
colors = ['green' if score >= 0 else 'red' for score in sentiment_by_topic_store.values]

# Create the horizontal bar plot
plt.figure(figsize=(10, 6))
sns.barplot(
    x=sentiment_by_topic_store.values,
    y=sentiment_by_topic_store.index,
    palette=colors
)

# Labels and title
plt.xlabel("Average Sentiment Score")
plt.ylabel("Tesco Store Theme")
plt.title("Sentiment Score per Topic – Tesco Stores")
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)  # Neutral reference line

# Clean up
plt.legend([], [], frameon=False)
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Count number of reviews per dominant topic
topic_distribution_store = df_store["Dominant Topic"].value_counts()

# Create a horizontal bar chart with blue bars
plt.figure(figsize=(10, 6))
sns.barplot(
    x=topic_distribution_store.values,
    y=topic_distribution_store.index,
    color="orange"
)

# Labels and title
plt.xlabel("Number of Reviews")
plt.ylabel("Tesco Store Theme")
plt.title("Distribution of Topics – Tesco Stores")

# Layout and show
plt.tight_layout()
plt.show()


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine all Tesco Stores reviews into a single string
all_reviews_store_text = " ".join(df_store["Review Message"].astype(str))

# Generate the word cloud
wordcloud_store = WordCloud(
    width=800,
    height=400,
    background_color="white",
    colormap="viridis"
).generate(all_reviews_store_text)

# Plot the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_store, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Tesco Store Reviews", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
def classify_sentiment(score):
    if score >= 0.3:
        return "Positive"
    elif score <= -0.1:
        return "Negative"
    else:
        return "Neutral"

df_store["Sentiment Category"] = df_store["Sentiment Score"].apply(classify_sentiment)


In [ ]:
# Recalculate sentiment distribution per topic
sentiment_distribution_store = (
    df_store.groupby("Dominant Topic")["Sentiment Category"]
    .value_counts(normalize=True)
    .unstack()
    .fillna(0)
)

# Ensure column order
sentiment_distribution_store = sentiment_distribution_store[["Negative", "Neutral", "Positive"]]

# Plot
sentiment_distribution_store.plot(
    kind="barh",
    stacked=True,
    color=["red", "lightgrey", "green"],  # manual mapping
    alpha=0.85,
    figsize=(12, 6)
)

plt.xlabel("Proportion of Sentiment")
plt.ylabel("Tesco Store Topic")
plt.title("Sentiment Distribution by Topic – Tesco Stores (Recalibrated)")
plt.legend(title="Sentiment Category")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Prepare matching topic order
topics_store = list(sentiment_by_topic_store.index)  # topics in sentiment order

# Ensure the same topic order for review counts
review_counts_store = np.array([df_store["Dominant Topic"].value_counts().get(topic, 0) for topic in topics_store])
sentiment_scores_store = np.array([sentiment_by_topic_store[topic] for topic in topics_store])

# Step 2: Create plot
fig, ax1 = plt.subplots(figsize=(12, 6))

# Bar positions
y_pos = np.arange(len(topics_store))

# Plot review counts (Blue bars)
ax1.barh(y_pos - 0.2, review_counts_store, color="lightblue", height=0.4, label="Number of Reviews")
ax1.set_xlabel("Number of Reviews", color="blue", fontsize=12)
ax1.set_ylabel("Tesco Store Topic", fontsize=12)
ax1.set_yticks(y_pos)
ax1.set_yticklabels(topics_store)
ax1.tick_params(axis="x", labelcolor="blue")

# Plot sentiment scores (Red bars on second x-axis)
ax2 = ax1.twiny()
ax2.barh(y_pos + 0.2, sentiment_scores_store, color="lightcoral", height=0.4, label="Average Sentiment")
ax2.set_xlabel("Average Sentiment Score", color="red", fontsize=12)
ax2.tick_params(axis="x", labelcolor="red")

# Title and legends
plt.title("Topic Distribution and Average Sentiment – Tesco Stores", fontsize=14)
ax1.legend(loc="lower right", fontsize=10)
ax2.legend(loc="upper right", fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Annotated dataset for 3-star reviews tesco bank
dfstore_annotations = pd.read_csv('/content/drive/MyDrive/MSc Dissertation/Tesco Stores Review (1).csv')  # should have columns like: review_text, annotated_sentiment


# Inspect the data
print(dfstore_annotations.head())


In [ ]:
df_store = df_store.merge(
    dfstore_annotations[['Review', 'Annotation']],
    left_on='Review Message',
    right_on='Review',
    how='left'
)


In [ ]:
df_store.drop(columns='Review', inplace=True)

In [ ]:
df_store.head()

In [ ]:
def define_expected_sentiment(row):
    if pd.notnull(row['Annotation']):
        return row['Annotation'].capitalize()
    elif row['Star Rating'] in [1, 2]:
        return 'Negative'
    elif row['Star Rating'] in [4, 5]:
        return 'Positive'
    else:
        return 'Neutral'  # For 3-star reviews without annotations


In [ ]:
df_store['Expected Sentiment'] = df_store.apply(define_expected_sentiment, axis=1)

In [ ]:
df_store.head()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Step 1: Set up ground truth and predicted labels
y_true = df_store['Expected Sentiment']
y_pred = df_store['Sentiment Category']  # VADER's predicted category

# Step 2: Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

# Step 3: Display metrics
print(f"✅ Accuracy: {accuracy:.2f}")
print(f"✅ Precision: {precision:.2f}")
print(f"✅ Recall: {recall:.2f}")
print(f"✅ F1 Score: {f1:.2f}")

# Step 4: Detailed classification report
print("\n📊 Classification Report:")
print(classification_report(
    y_true,
    y_pred,
    labels=["Negative", "Neutral", "Positive"],
    target_names=["Negative", "Neutral", "Positive"],
    zero_division=0
))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

labels = ["Negative", "Neutral", "Positive"]
cm = confusion_matrix(y_true, y_pred, labels=labels)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted Sentiment")
plt.ylabel("True (Expected) Sentiment")
plt.title("Confusion Matrix – VADER vs Expected Sentiment (Tesco Stores)")
plt.tight_layout()
plt.show()


In [ ]:
!pip install transformers torch emoji

In [ ]:
import re

# Define lightweight cleaning function
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # Remove URLs
    text = re.sub(r'\s+', ' ', text).strip()             # Remove extra whitespace
    text = text.replace('\n', ' ').replace('\r', ' ')    # Flatten line breaks
    return text

# Apply to Tesco Store dataset
df_store['Review Message'] = df_store['Review Message'].apply(clean_text)


In [ ]:
def batch_bert_sentiment(texts, batch_size=32):
    sentiments = []
    model.eval()

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=1)
            preds = torch.argmax(probs, dim=1)

        batch_sentiments = [labels[p] for p in preds]
        sentiments.extend(batch_sentiments)

    return sentiments


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load tokenizer and model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Force use of CPU
device = torch.device('cpu')
model.to(device)

# Define label map
labels = ['Negative', 'Neutral', 'Positive']


In [ ]:
df_store['RoBERTa Sentiment'] = batch_bert_sentiment(df_store['Review Message'].tolist())

In [ ]:
import torch
torch.cuda.is_available()


In [ ]:
def batch_bert_sentiment(texts, batch_size=8):  # reduce batch size
    sentiments = []
    model.eval()

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        print(f"Processing batch {i}-{i+batch_size}...")  # track progress

        try:
            inputs = tokenizer(
                batch_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            ).to(device)

            with torch.no_grad():
                outputs = model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=1)
                preds = torch.argmax(probs, dim=1)

            batch_sentiments = [labels[p] for p in preds]
            sentiments.extend(batch_sentiments)

        except Exception as e:
            print(f"⚠️ Error in batch {i}-{i+batch_size}: {e}")
            sentiments.extend([None]*len(batch_texts))

    return sentiments


In [ ]:
df_test = df_store.head(50).copy()
df_test['RoBERTa Sentiment'] = batch_bert_sentiment(df_test['Review Message'].tolist())


In [ ]:
# Store predictions here
roberta_sentiments = []

# Define batch size (adjust if needed)
batch_size = 64

# Total number of reviews
texts = df_store['Review Message'].dropna().tolist()
total = len(texts)

# Process in batches
for i in range(0, total, batch_size):
    print(f"Processing batch {i} to {min(i + batch_size, total)}...")
    batch_texts = texts[i:i+batch_size]
    batch_preds = batch_bert_sentiment(batch_texts)
    roberta_sentiments.extend(batch_preds)


In [ ]:
# Add the results as a new column
df_store = df_store.copy()
df_store.loc[:len(roberta_sentiments)-1, 'RoBERTa Sentiment'] = roberta_sentiments


In [ ]:
df_store[['Review Message', 'Expected Sentiment', 'RoBERTa Sentiment']].head()


In [ ]:
def batch_bert_sentiment_scores(texts, batch_size=32):
    all_scores = []
    model.eval()

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        print(f"Processing batch {i}-{i+batch_size}...")

        try:
            inputs = tokenizer(
                batch_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            ).to(device)

            with torch.no_grad():
                outputs = model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=1)

            # Assign sentiment score: Negative = -1, Neutral = 0, Positive = +1
            weights = torch.tensor([-1, 0, 1], device=device).float()
            scores = (probs * weights).sum(dim=1).tolist()  # weighted average

            all_scores.extend(scores)

        except Exception as e:
            print(f"⚠️ Error in batch {i}-{i+batch_size}: {e}")
            all_scores.extend([None] * len(batch_texts))

    return all_scores


In [ ]:
# Run it on all reviews
sentiment_score_list = batch_bert_sentiment_scores(df_store['Review Message'].tolist())

# Add to df_store
df_store['RoBERTa Sentiment Score'] = sentiment_score_list


In [ ]:
df_store.head()

In [ ]:
# Group by Dominant Topic and calculate average RoBERTa sentiment score
avg_sentiment_per_topic = df_store.groupby("Dominant Topic")["RoBERTa Sentiment Score"].mean().sort_values()

# Display as a neat table
import pandas as pd
pd.set_option('display.float_format', '{:.3f}'.format)  # format float nicely
avg_sentiment_per_topic = avg_sentiment_per_topic.reset_index()
avg_sentiment_per_topic.columns = ['Topic', 'Average RoBERTa Sentiment Score']
avg_sentiment_per_topic


In [ ]:
# Evaluate RoBERTa predictions
y_true = df_store['Expected Sentiment']
y_pred = df_store['RoBERTa Sentiment']

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print(f"✅ RoBERTa Accuracy: {accuracy:.2f}")
print(f"✅ RoBERTa Precision: {precision:.2f}")
print(f"✅ RoBERTa Recall: {recall:.2f}")
print(f"✅ RoBERTa F1 Score: {f1:.2f}")

print("\n📊 RoBERTa Classification Report:")
print(classification_report(
    y_true,
    y_pred,
    labels=["Negative", "Neutral", "Positive"],
    target_names=["Negative", "Neutral", "Positive"],
    zero_division=0
))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Define labels for consistency
labels = ['Negative', 'Neutral', 'Positive']

# Create confusion matrix
cm = confusion_matrix(df_store['Expected Sentiment'], df_store['RoBERTa Sentiment'], labels=labels)

# Plot heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="YlGnBu", xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted Sentiment (RoBERTa)")
plt.ylabel("Actual Sentiment (Expected)")
plt.title("Confusion Matrix – RoBERTa vs Expected Sentiment - Tesco Stores")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Recalculate average sentiment per topic
avg_sentiment = df_store.groupby("Dominant Topic")["RoBERTa Sentiment Score"].mean().sort_values()

# Define colors based on score
colors = ['green' if val > 0 else 'red' for val in avg_sentiment]

# Plot with conditional colors
plt.figure(figsize=(10, 6))
avg_sentiment.plot(kind="barh", color=colors)
plt.xlabel("Average RoBERTa Sentiment Score")
plt.ylabel("Tesco Store Topic")
plt.title("Average Sentiment Score per Topic – Tesco Stores")
plt.axvline(0, color='black', linestyle='--')
plt.tight_layout()
plt.show()


In [ ]:
# Step 2: Ensure column order
sentiment_distribution_store = sentiment_distribution_store[["Negative", "Neutral", "Positive"]]

# Step 3: Plot stacked horizontal bar chart
sentiment_distribution_store.plot(
    kind="barh",
    stacked=True,
    color=["red", "lightgrey", "green"],  # Custom sentiment colors
    alpha=0.85,
    figsize=(12, 6)
)

plt.xlabel("Proportion of Sentiment")
plt.ylabel("Tesco Store Topic")
plt.title("Sentiment Distribution by Topic – Tesco Stores (RoBERTa)")
plt.legend(title="Sentiment Category", loc="lower right")
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Make sure average sentiment per topic is calculated
sentiment_by_topic_store = df_store.groupby("Dominant Topic")["RoBERTa Sentiment Score"].mean().sort_values()

# Step 2: Match topic order across both axes
topics_store = list(sentiment_by_topic_store.index)
review_counts_store = np.array([df_store["Dominant Topic"].value_counts().get(topic, 0) for topic in topics_store])
sentiment_scores_store = np.array([sentiment_by_topic_store[topic] for topic in topics_store])

# Step 3: Create dual-axis plot
fig, ax1 = plt.subplots(figsize=(12, 6))
y_pos = np.arange(len(topics_store))

# Left axis: Number of reviews (blue bars)
ax1.barh(y_pos - 0.2, review_counts_store, color="lightblue", height=0.4, label="Number of Reviews")
ax1.set_xlabel("Number of Reviews", color="blue", fontsize=12)
ax1.set_ylabel("Tesco Store Topic", fontsize=12)
ax1.set_yticks(y_pos)
ax1.set_yticklabels(topics_store)
ax1.tick_params(axis="x", labelcolor="blue")

# Right axis: Average sentiment score (red bars)
ax2 = ax1.twiny()
ax2.barh(y_pos + 0.2, sentiment_scores_store, color="lightcoral", height=0.4, label="Average Sentiment")
ax2.set_xlabel("Average Sentiment Score", color="red", fontsize=12)
ax2.tick_params(axis="x", labelcolor="red")

# Add title and legends
plt.title("Topic Distribution and Average Sentiment – Tesco Stores", fontsize=14)
ax1.legend(loc="lower right", fontsize=10)
ax2.legend(loc="upper right", fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
# Filter negative reviews
df_negative_store = df_store[df_store['Star Rating'].isin([1, 2, 3])].copy()

# If you have trigrammed tokens for the full dataset, do the same for negatives
texts_trigrams_neg = df_negative_store["cleaned_text"]


In [ ]:
texts_lemmatized_neg = lemmatize_texts(texts_trigrams_neg)

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel

# Create Dictionary and Corpus
id2word_neg = Dictionary(texts_lemmatized_neg)
corpus_neg = [id2word_neg.doc2bow(text) for text in texts_lemmatized_neg]

# Train LDA model
lda_model_neg = LdaModel(
    corpus=corpus_neg,
    id2word=id2word_neg,
    num_topics=3,  # You can tune this!
    random_state=42,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
)


In [ ]:
for idx, topic in lda_model_neg.print_topics(num_words=10):
    print(f"Topic {idx}: {topic}")

In [ ]:
from gensim.models import CoherenceModel

# Build coherence model
coherence_model_neg = CoherenceModel(
    model=lda_model_neg,
    texts=texts_lemmatized_neg,
    dictionary=id2word_neg,
    coherence='c_v'  # You can also try 'u_mass' or 'c_uci' for comparison
)

# Compute the coherence score
coherence_score = coherence_model_neg.get_coherence()
print(f"\n✅ Coherence Score (c_v): {coherence_score:.4f}")


In [ ]:
negative_themes = {
0: "Delivery and Order",
1: "In-Store Experience",
2: "Product and Pricing"
}

In [ ]:
# Assign dominant topic using the LDA model for negative reviews
def get_dominant_topic(ldamodel, corpus):
    dominant_topics = []
    for bow in corpus:
        topics = ldamodel.get_document_topics(bow)
        if topics:
            dominant_topic = max(topics, key=lambda x: x[1])[0]
        else:
            dominant_topic = None
        dominant_topics.append(dominant_topic)
    return dominant_topics

# Assign topics to the negative dataset
df_negative_store["LDA Topic"] = get_dominant_topic(lda_model_neg, corpus_neg)


In [ ]:
# Map LDA Topic to human-readable theme
df_negative_store["Complaint Theme"] = df_negative_store["LDA Topic"].map(negative_themes)

# Compute average sentiment score per complaint theme
avg_sentiment_per_theme = (
    df_negative_store.groupby("Complaint Theme")["RoBERTa Sentiment Score"]
    .mean()
    .sort_values()
    .reset_index()
)

# Display table
avg_sentiment_per_theme.columns = ['Complaint Theme', 'Average Sentiment Score']
avg_sentiment_per_theme.style.format({'Average Sentiment Score': '{:.4f}'})
